In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu

In [3]:
job_params = {
    'subjects_dir' :   '/mnt/a2',
    'subject' :        'AH007',
    'date' :           '2022-03-28',
    'expnum' :         [2,4,5],
    'planes' :         n.arange(0,30),
    
    'notch_filt' :     None, #{'f0' : 200, 'Q' : 1},
    
    'n_init_files' :   3,
    'init_file_pool' : None,
    'init_file_sample_method': 'random',
    
    'subtract_crosstalk' : True,
    'override_crosstalk' : None,
    'crosstalk_n_planes' : 10, 
    
    'nonrigid' : True,
    'smooth_sigma' : 1.15,
    'maxregshift' : 0.15,
    'reg_filter_pcorr' : 1,
    'generate_sample_registered_bins' : False,

    'dtype' : n.float32,
    'tif_batch_size' : 1,
}



In [4]:
tifs, si_params, exp_str = flu.find_exp(job_params['subjects_dir'], job_params['subject'],
                                        job_params['date'], job_params['expnum'])

SI ROI pix count in x is 182, which is impossible, setting it to 145


In [5]:
job_params['fs'] = si_params['vol_rate']

In [6]:
# Create the job
job = Job('/mnt/md0/runs',exp_str, tifs = tifs,
          params=job_params, create=False, overwrite=False, verbosity = 3)

   Found and loaded params from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/params.npy


In [7]:
# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files'] = 3
job.params['init_n_frames'] = None


In [8]:
# %%time
# summary = job.run_init_pass()

In [9]:
# split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [10]:
# %%time
# job.register_gpu()

In [56]:
mov_full = job.get_registered_movie('registered_fused_data', 'fused')
# im_full = mov_full[:,:200].mean(axis=1).compute()

In [12]:
# find crop values that minimize dark zones - check planes 0 and 15 in the following cell to 
# make sure you're not cutting out parts of the brain
crop = ((0,18), (100,1100), (50, 900))

In [13]:
# tfu.show_tif(im_full[0,crop[1][0]:crop[1][1], crop[2][0]:crop[2][1]])
# tfu.show_tif(im_full[15,crop[1][0]:crop[1][1], crop[2][0]:crop[2][1]])

In [14]:
# tag (not path) for the directory where the SVD will be saved
svd_dir_tag = 'svd_cropped'

# Number of components to compute per block 
# (you can change the actual number of components used for computation later, this is the upper limit)
job.params['n_svd_comp'] = 100
# Size of each block in pixels in z,y,x
job.params['svd_block_shape'] = (4,200,200)
# overlap in z,y,x between two neighboring blocks
job.params['svd_block_overlaps'] = (2,100,100)
# crop the movie before computing svd
job.params['svd_crop'] = crop
# When computing SVD, we can compute multiple blocks (4-8) at the same time, which is sometimes 
# faster since we save on some disk I/O for neighboring blocks (I  think)
# for longer recordings (1000+frames) or if you have issues with RAM, set to 1
job.params['n_svd_blocks_per_batch'] = 1

In [17]:
job.params['svd_time_crop'] = None # (0,5000)
job.params['svd_pix_chunk'] = n.product(job.params['svd_block_shape']) // 2
job.params['svd_time_chunk'] = 1024

In [ ]:
%%time
# create the directory where we'll save the SVD, and run the SVD decomposition
job.make_new_dir(svd_dir_tag)
svd_info = job.svd_decompose_movie(svd_dir_tag, run_svd=True)

      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped with tag svd_cropped
   Saved a copy of params at /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped
   Updated main params file
   Loaded mov of size (30, 10194, 1255, 973)
   Cropped to size (18, 10194, 1000, 850)
   Saved a copy of params at /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped
   Updated main params file
576 blocks with overlap (125 without, 4.61x increase)
   Will compute SVD in 576 blocks in a grid shaped (8, 9, 8)
   Batching 1 blocks together, for a total of 576 batches. Pix chunksize: 80000
   Saving SVD blocks in /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/blocks
Starting with batch 0
      Starting batch 1 / 576, blocks 0 - 1
      Sending batch 1 to dask
      Dask completed in 128.716 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 1 completed in 128.770
   Estimated time remaining for 575 batches: 20h34m03s
      Starting 

      Dask completed in 112.564 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 27 completed in 112.603
   Estimated time remaining for 549 batches: 17h00m55s
      Starting batch 28 / 576, blocks 27 - 28
      Sending batch 28 to dask
      Dask completed in 106.480 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 28 completed in 106.526
   Estimated time remaining for 548 batches: 16h22m09s
      Starting batch 29 / 576, blocks 28 - 29
      Sending batch 29 to dask
      Dask completed in 109.961 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 29 completed in 110.007
   Estimated time remaining for 547 batches: 16h38m23s
      Starting batch 30 / 576, blocks 29 - 30
      Sending batch 30 to dask
      Dask completed in 107.727 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.n

      Dask completed in 107.704 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 56 completed in 107.760
   Estimated time remaining for 520 batches: 15h36m12s
      Starting batch 57 / 576, blocks 56 - 57
      Sending batch 57 to dask
      Dask completed in 106.484 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 57 completed in 106.531
   Estimated time remaining for 519 batches: 15h24m04s
      Starting batch 58 / 576, blocks 57 - 58
      Sending batch 58 to dask
      Dask completed in 108.507 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 58 completed in 108.549
   Estimated time remaining for 518 batches: 15h34m10s
      Starting batch 59 / 576, blocks 58 - 59
      Sending batch 59 to dask
      Dask completed in 107.631 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.n

      Dask completed in 108.423 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 85 completed in 108.473
   Estimated time remaining for 491 batches: 14h46m21s
      Starting batch 86 / 576, blocks 85 - 86
      Sending batch 86 to dask
      Dask completed in 110.189 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 86 completed in 110.240
   Estimated time remaining for 490 batches: 14h57m08s
      Starting batch 87 / 576, blocks 86 - 87
      Sending batch 87 to dask
      Dask completed in 108.538 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 87 completed in 108.595
   Estimated time remaining for 489 batches: 14h47m06s
      Starting batch 88 / 576, blocks 87 - 88
      Sending batch 88 to dask
      Dask completed in 107.742 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.n

      Dask completed in 106.394 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 113 completed in 106.451
   Estimated time remaining for 463 batches: 13h45m06s
      Starting batch 114 / 576, blocks 113 - 114
      Sending batch 114 to dask
      Dask completed in 107.580 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 114 completed in 107.630
   Estimated time remaining for 462 batches: 13h47m40s
      Starting batch 115 / 576, blocks 114 - 115
      Sending batch 115 to dask
      Dask completed in 109.638 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 115 completed in 111.911
   Estimated time remaining for 461 batches: 14h13m03s
      Starting batch 116 / 576, blocks 115 - 116
      Sending batch 116 to dask
      Dask completed in 108.749 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cro

...

      Dask completed in 109.237 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 197 completed in 109.291
   Estimated time remaining for 379 batches: 11h27m32s
      Starting batch 198 / 576, blocks 197 - 198
      Sending batch 198 to dask
      Dask completed in 108.186 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 198 completed in 108.238
   Estimated time remaining for 378 batches: 11h22m39s
      Starting batch 199 / 576, blocks 198 - 199
      Sending batch 199 to dask
      Dask completed in 110.778 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 199 completed in 110.823
   Estimated time remaining for 377 batches: 11h33m14s
      Starting batch 200 / 576, blocks 199 - 200
      Sending batch 200 to dask
      Dask completed in 108.773 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cro

      Dask completed in 107.670 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 225 completed in 107.727
   Estimated time remaining for 351 batches: 10h29m18s
      Starting batch 226 / 576, blocks 225 - 226
      Sending batch 226 to dask
      Dask completed in 106.634 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 226 completed in 106.684
   Estimated time remaining for 350 batches: 10h23m21s
      Starting batch 227 / 576, blocks 226 - 227
      Sending batch 227 to dask
      Dask completed in 107.687 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 227 completed in 107.744
   Estimated time remaining for 349 batches: 10h25m41s
      Starting batch 228 / 576, blocks 227 - 228
      Sending batch 228 to dask
      Dask completed in 107.313 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cro

      Dask completed in 107.387 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 253 completed in 107.439
   Estimated time remaining for 323 batches: 09h39m39s
      Starting batch 254 / 576, blocks 253 - 254
      Sending batch 254 to dask
      Dask completed in 108.659 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 254 completed in 108.703
   Estimated time remaining for 322 batches: 09h42m16s
      Starting batch 255 / 576, blocks 254 - 255
      Sending batch 255 to dask
      Dask completed in 106.811 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 255 completed in 106.867
   Estimated time remaining for 321 batches: 09h33m28s
      Starting batch 256 / 576, blocks 255 - 256
      Sending batch 256 to dask
      Dask completed in 109.163 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cro

      Dask completed in 106.919 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 281 completed in 106.968
   Estimated time remaining for 295 batches: 08h46m21s
      Starting batch 282 / 576, blocks 281 - 282
      Sending batch 282 to dask
      Dask completed in 110.173 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 282 completed in 112.318
   Estimated time remaining for 294 batches: 09h05m12s
      Starting batch 283 / 576, blocks 282 - 283
      Sending batch 283 to dask
      Dask completed in 107.640 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 283 completed in 107.697
   Estimated time remaining for 293 batches: 08h49m24s
      Starting batch 284 / 576, blocks 283 - 284
      Sending batch 284 to dask
      Dask completed in 111.387 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cro

      Dask completed in 107.644 sec
      Saving svd_info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/svd_cropped/svd_info.npy
   Batch 309 completed in 107.694
   Estimated time remaining for 267 batches: 07h59m37s
      Starting batch 310 / 576, blocks 309 - 310
      Sending batch 310 to dask


In [18]:
svd_info = n.load(os.path.join(job.dirs[svd_dir_tag], 'svd_info.npy'),allow_pickle=True).item()

In [51]:
svd_info_from_demo = n.load('/mnt/md0/runs/s3d-Full-Demo/svd_cropped/svd_info.npy',allow_pickle=True).item()

In [74]:
# number of SVD components to use when calculating the correlation map
# lower number means more denoising, but if it's too low you will start losing cells!
# This can't be larger than the number of svd components you used in the decomposition above
job.params['n_svd_comp'] = 50

# spatial filter sizes for neuropil subtraction, and cell detection
# npil_filt is a low_pass filter that attempts to remove any features larger than the filter size (neuropil!)
# conv_filt_xy is a high_pass filter that amplifies any features that are smaller than ~2x the filter size (cells!)
# these values worked well for me with ~4um xy pixel spacing and ~15 um z pixel spacing, for detecting mouse somata
# When you change resolution, or if you're trying to detect smaller things, you will need to adjust these values
# because the units here are _pixels_, not microns!
job.params['conv_filt_type'] = 'gaussian'
job.params['conv_filt_xy'] = 1.0
job.params['conv_filt_z'] = 1.0
job.params['npil_filt_type'] = 'unif'
job.params['npil_filt_xy'] = 8.0
job.params['npil_filt_z']=  1.5

# normalization exponent, should be around 1. 
# If you find blood vessels or the background being too bright in the correlation map, reduce it to ~0.7-0.8! 
job.params['sdnorm_exp']= 0.8

# threshold applied to the normalized, filtered movie before it is accumulated into the correlation map
# if you increase it, the background will become darker (which is good!), however at some point you will
# start excluding dimmer cells (which is bad!)
job.params['intensity_thresh'] : 0.4

## Compute parameters 
# number of frames to compute at one iteration 
# (any value above ~100-200 shouldn't affect results, 
# decrease if you have RAM issues or if SVD reconstruction gets stuck on "Sending all blocks to dask to compute")
job.params['t_batch_size'] = 300
# number of processors to use when calculating the correlation map
job.params['n_proc_corr'] = 70
# number of frames per smaller batch within the batch, should be ~t_batch_size / n_proc_corr, but above ~5
job.params['mproc_batchsize'] = 5


In [75]:
mov_crop = mov_full[crop[0][0]:crop[0][1], :, crop[1][0]:crop[1][1], crop[2][0]:crop[2][1]]

In [76]:
%%time
# uncomment below to load svd_info for and svd you did earlier if you are re-running this notebook
# svd_info = n.load(os.path.join(job.dirs['svd_cropped'], 'svd_info.npy'), allow_pickle=True).item()
(vmap, mean_img, max_img), __, __ = job.calculate_corr_map(mov = mov_crop)

   Updated main params file
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/mov_sub with tag mov_sub
   Saving mov_sub to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/mov_sub
   Using conv_filt: gaussian, 1.00, 1.00
   Using np_filt: unif, 1.50, 8.00
   Created files and dirs for 34 batches
      Running batch 1 of 34
         Will process 300 frames (0-300, t_batch_size: 300)
      Loaded and swapped, idx 0 to 300
      Calculating corr map
         Rolling mean filter
         Stdev over time
         Sharr creation
         Sub and conv
         Vmap
   Calculated corr map in 132.83 seconds
      Saving to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0000
      Running batch 2 of 34
         Will process 300 frames (300-600, t_batch_size: 300)
      Loaded and swapped, idx 300 to 600
      Calculating corr map
         Rolling mean filter
         Stdev over time
         Sharr creation
         Sub and conv
         Vmap
   Calculated corr map in 156.95 seconds
      Sa

         Stdev over time
         Sharr creation
         Sub and conv
         Vmap
   Calculated corr map in 185.31 seconds
      Saving to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0020
      Running batch 22 of 34
         Will process 300 frames (6300-6600, t_batch_size: 300)
      Loaded and swapped, idx 6300 to 6600
      Calculating corr map
         Rolling mean filter
         Stdev over time
         Sharr creation
         Sub and conv
         Vmap
   Calculated corr map in 196.52 seconds
      Saving to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0021
      Running batch 23 of 34
         Will process 300 frames (6600-6900, t_batch_size: 300)
      Loaded and swapped, idx 6600 to 6900
      Calculating corr map
         Rolling mean filter
         Stdev over time
         Sharr creation
         Sub and conv
         Vmap
   Calculated corr map in 194.66 seconds
      Saving to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0022
      Running batch

In [80]:
napari.view_image(vmap)

Viewer(camera=Camera(center=(0.0, 499.5, 424.5), zoom=1.1386718111665908, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(8.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 18.0, 1.0), (0.0, 1000.0, 1.0), (0.0, 850.0, 1.0)), current_step=(8, 499, 424), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'vmap' at 0x7f2d718debb0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x7f30c89b1310>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [77]:
# # Pick the parameters you want to sweep, and enter them in the tuples.
# # It will do all combinations of parameters, so if you have many parameters it will be many combinations! 
# job.params['t_batch_size'] = 300
# params_to_sweep = {
#     'intensity_thresh' : (0.25, 0.4),
#     'n_svd_comp' : (50,), # if you have multiple values here, make sure you pass mov=svd_info
#     'conv_filt_xy': (0.7,1.0,),
#     'npil_filt_xy': (3.0,5.0,8.0),
#     'conv_filt_z' : (1.0,1.5),
#     'npil_filt_z' : (1.0,1.5,),
#     'sdnorm_exp' : (0.8, 1.0)
# }

# sweep_summary = job.sweep_params(params_to_sweep, mov = mov_crop,testing_dir_tag='sweep-nosvd',
#                            do_vmap=True, all_combinations=True, n_test_iters = 1)
# # use this to access older sweep results
# # sweep_summary = n.load(os.path.join(job.dirs['sweep-full'], 'sweep_summary.npy'),allow_pickle=True).item()
# job.vis_vmap_sweep(sweep_summary)

In [93]:
%%time
job.params['detection_timebin'] = 1 
job.params['detection_time_crop'] = (None,None)
job.params['max_pix'] = 250
job.params['peak_thresh'] = 24.4

job.params['activity_thresh'] = 10.0
job.params['percentile'] = 99.5
job.params['extend_thresh'] = 0.15
job.params['max_iter'] = 10000 # maximum number of ROIs detected in given patch
combined_dir = job.patch_and_detect(combined_name='combined-full')

   Loading from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0033
99 blocks with overlap (72 without, 1.38x increase)
99 blocks with overlap (72 without, 1.38x increase)
   Detecting from patch: 0/99
   Updated main params file
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection with tag detection
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0000 with tag patch-0000
   Running cell detection on patch 0000 at /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0000, max 10000 iters
   Patch bounds are [ 0 18], [  0 120], [  0 120]
   Cell center bounds are [ 0 18], [  0 104], [  0 105]
   Time bounds are (None, None)
mov_sub
   Loading 9.84 GB movie to memory
   Loaded
   Loading from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0033
      Movie shape: (10194, 18, 120, 120)
   Saving cell stats and info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0000
   Loading movie patch to shared memory
   Starting extract

   Loaded
   Loading from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0033
      Movie shape: (10194, 18, 120, 120)
   Saving cell stats and info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0005
   Loading movie patch to shared memory
   Starting extraction with peak_thresh: 24.400 and Th2: 10.000
      Iter 0000: peak is too small  - ending extraction
   Found 0 cells in 0 iterations
   Saving cells to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0005/stats.npy
   Saving iscell.npy to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0005/iscell.npy
   Detecting from patch: 6/99
   Updated main params file
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection with tag detection
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0006 with tag patch-0006
   Running cell detection on patch 0006 at /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0006, max 10000 iters
   Patch bounds are [ 0 18], [  0 12

      Added cell 17 at 05, 149, 281, peak: 31.627, thresh: 8.874, 128 frames, 37 pixels
      Added cell 18 at 00, 142, 281, peak: 30.778, thresh: 8.254, 123 frames, 34 pixels
      Added cell 19 at 04, 146, 255, peak: 30.709, thresh: 7.832, 184 frames, 42 pixels
      Added cell 20 at 02, 135, 249, peak: 30.518, thresh: 7.720, 166 frames, 60 pixels
      Added cell 21 at 02, 189, 272, peak: 30.328, thresh: 8.110, 196 frames, 64 pixels
      Added cell 22 at 12, 158, 236, peak: 29.810, thresh: 7.690, 248 frames, 44 pixels
      Added cell 23 at 02, 120, 248, peak: 29.570, thresh: 7.326, 155 frames, 48 pixels
      Added cell 24 at 06, 190, 222, peak: 29.475, thresh: 7.865, 102 frames, 22 pixels
   Iter 0003: running 08 ROIs in parallel
      Added cell 25 at 00, 163, 255, peak: 29.286, thresh: 8.343, 84 frames, 21 pixels
      Added cell 26 at 07, 141, 286, peak: 28.928, thresh: 8.006, 117 frames, 36 pixels
      Added cell 27 at 13, 104, 272, peak: 28.395, thresh: 6.624, 76 frames, 15

   Found 107 cells in 14 iterations
   Saving cells to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0011/stats.npy
   Saving iscell.npy to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0011/iscell.npy
   Detecting from patch: 12/99
   Updated main params file
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection with tag detection
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0012 with tag patch-0012
   Running cell detection on patch 0012 at /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0012, max 10000 iters
   Patch bounds are [ 0 18], [ 88 208], [273 393]
   Cell center bounds are [ 0 18], [104 192], [287 379]
   Time bounds are (None, None)
mov_sub
   Loading 9.84 GB movie to memory
   Loaded
   Loading from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/iters/batch0033
      Movie shape: (10194, 18, 120, 120)
   Saving cell stats and info to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0012
   Loading 

      Added cell 81 at 07, 171, 300, peak: 27.445, thresh: 8.018, 134 frames, 47 pixels
      Added cell 82 at 04, 144, 364, peak: 27.411, thresh: 7.759, 75 frames, 17 pixels
      Added cell 83 at 11, 152, 294, peak: 27.389, thresh: 7.106, 212 frames, 30 pixels
      Added cell 84 at 04, 164, 376, peak: 27.361, thresh: 7.874, 85 frames, 35 pixels
      Added cell 85 at 05, 131, 359, peak: 27.349, thresh: 7.428, 120 frames, 52 pixels
      Added cell 86 at 01, 141, 310, peak: 27.321, thresh: 7.218, 108 frames, 43 pixels
      Added cell 87 at 01, 145, 328, peak: 27.306, thresh: 7.672, 92 frames, 25 pixels
      Added cell 88 at 08, 139, 347, peak: 27.252, thresh: 7.441, 96 frames, 20 pixels
   Iter 0011: running 08 ROIs in parallel
      Added cell 89 at 00, 178, 309, peak: 27.243, thresh: 7.605, 64 frames, 16 pixels
      Added cell 90 at 04, 185, 300, peak: 27.234, thresh: 6.899, 175 frames, 67 pixels
      Added cell 91 at 07, 153, 297, peak: 27.183, thresh: 8.051, 107 frames, 36 pi

...

      Added cell 9 at 02, 929, 654, peak: 29.659, thresh: 7.138, 155 frames, 39 pixels
      Added cell 10 at 05, 925, 654, peak: 29.632, thresh: 6.958, 257 frames, 62 pixels
      Added cell 11 at 08, 919, 654, peak: 29.133, thresh: 7.505, 152 frames, 52 pixels
      Added cell 12 at 07, 947, 660, peak: 29.072, thresh: 7.764, 178 frames, 46 pixels
      Added cell 13 at 03, 913, 664, peak: 28.972, thresh: 7.625, 136 frames, 34 pixels
      Added cell 14 at 00, 915, 668, peak: 28.966, thresh: 7.735, 91 frames, 14 pixels
      Added cell 15 at 05, 942, 683, peak: 28.574, thresh: 7.234, 81 frames, 16 pixels
      Added cell 16 at 04, 947, 676, peak: 28.421, thresh: 7.394, 144 frames, 36 pixels
   Iter 0002: running 08 ROIs in parallel
      Added cell 17 at 02, 925, 658, peak: 28.794, thresh: 7.221, 140 frames, 43 pixels
      Added cell 18 at 01, 910, 668, peak: 28.624, thresh: 7.604, 125 frames, 33 pixels
      Added cell 19 at 08, 927, 703, peak: 28.383, thresh: 7.885, 110 frames, 21 

      Added cell 105 at 01, 897, 657, peak: 25.414, thresh: 6.913, 67 frames, 9 pixels
      Added cell 106 at 02, 914, 669, peak: 25.394, thresh: 7.009, 64 frames, 15 pixels
      Added cell 107 at 04, 899, 662, peak: 25.373, thresh: 7.075, 68 frames, 12 pixels
      Added cell 108 at 02, 904, 675, peak: 25.370, thresh: 7.260, 72 frames, 13 pixels
      Added cell 109 at 02, 903, 652, peak: 25.359, thresh: 6.942, 69 frames, 22 pixels
      Added cell 110 at 07, 896, 671, peak: 25.347, thresh: 7.534, 65 frames, 9 pixels
      Added cell 111 at 03, 942, 684, peak: 25.316, thresh: 6.964, 79 frames, 13 pixels
      Added cell 112 at 04, 936, 668, peak: 25.304, thresh: 6.760, 68 frames, 12 pixels
   Iter 0014: running 08 ROIs in parallel
      Added cell 113 at 00, 896, 661, peak: 25.386, thresh: 7.297, 75 frames, 10 pixels
      Added cell 114 at 03, 899, 675, peak: 25.333, thresh: 7.245, 73 frames, 13 pixels
      Added cell 115 at 04, 914, 673, peak: 25.294, thresh: 7.033, 91 frames, 8 

      Added cell 201 at 01, 900, 681, peak: 24.800, thresh: 7.389, 61 frames, 10 pixels
      Added cell 202 at 06, 904, 664, peak: 24.797, thresh: 7.219, 59 frames, 8 pixels
      Added cell 203 at 00, 903, 665, peak: 24.789, thresh: 7.030, 57 frames, 10 pixels
      Added cell 204 at 03, 922, 666, peak: 24.783, thresh: 6.939, 72 frames, 9 pixels
      Added cell 205 at 03, 909, 661, peak: 24.778, thresh: 6.858, 69 frames, 11 pixels
      Added cell 206 at 00, 924, 671, peak: 24.763, thresh: 7.058, 71 frames, 12 pixels
      Added cell 207 at 10, 917, 680, peak: 24.759, thresh: 6.880, 90 frames, 22 pixels
      Added cell 208 at 06, 906, 681, peak: 24.758, thresh: 7.037, 71 frames, 9 pixels
   Iter 0026: running 08 ROIs in parallel
      Added cell 209 at 07, 900, 661, peak: 24.791, thresh: 7.183, 66 frames, 12 pixels
      Added cell 210 at 02, 925, 662, peak: 24.765, thresh: 6.795, 68 frames, 15 pixels
      Added cell 211 at 02, 907, 684, peak: 24.755, thresh: 7.385, 73 frames, 9 p

      Added cell 297 at 03, 934, 668, peak: 24.426, thresh: 6.982, 54 frames, 12 pixels
      Added cell 298 at 02, 914, 658, peak: 24.423, thresh: 6.809, 63 frames, 9 pixels
      Added cell 299 at 01, 912, 675, peak: 24.418, thresh: 6.969, 63 frames, 11 pixels
      Added cell 300 at 03, 915, 671, peak: 24.418, thresh: 7.230, 57 frames, 10 pixels
      Added cell 301 at 04, 934, 684, peak: 24.406, thresh: 6.880, 75 frames, 10 pixels
      Added cell 302 at 09, 899, 684, peak: 24.405, thresh: 7.483, 55 frames, 11 pixels
      Added cell 303 at 05, 925, 704, peak: 24.404, thresh: 7.727, 77 frames, 9 pixels
      Iter 0038: peak is too small  - ending extraction
   Found 303 cells in 38 iterations
   Saving cells to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0097/stats.npy
   Saving iscell.npy to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0097/iscell.npy
   Detecting from patch: 98/99
   Updated main params file
      Found dir /mnt/md0/runs/s3d-AH007_2022-03

      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0057 with tag detection-patch-0057
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0058 with tag detection-patch-0058
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0059 with tag detection-patch-0059
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0060 with tag detection-patch-0060
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0061 with tag detection-patch-0061
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0062 with tag detection-patch-0062
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0063 with tag detection-patch-0063
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0064 with tag detection-patch-0064
      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/patch-0065 with tag detection-patch-0065
      Found dir /mnt/md0/run

In [92]:
outputs = ui.load_outputs(combined_dir, load_traces=False)
v,layers = ui.create_ui(outputs)

In [94]:
job.compute_npil_masks(combined_dir)

'/mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/combined-full'

In [95]:
traces = job.extract_and_deconvolve(stats_dir = combined_dir)

   Updated main params file
   Cropping with bounds: ((0, 18), (100, 1100), (50, 900))
   Movie shape: (18, 10194, 1000, 850)
22418
   Extracting 22418 valid cells, and saving cell flags to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/combined-full/iscell_extracted.npy
   Extracting activity
Will extract in 21 batches of 500
Saving intermediate results to /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/combined-full
Extracting batch 0000 of 0021
Batch size: 28 GB
Extracting batch 0001 of 0021
Batch size: 28 GB
Extracting batch 0002 of 0021
Batch size: 28 GB
Extracting batch 0003 of 0021
Batch size: 28 GB
Extracting batch 0004 of 0021
Batch size: 28 GB
Extracting batch 0005 of 0021
Batch size: 28 GB
Extracting batch 0006 of 0021
Batch size: 28 GB
Extracting batch 0007 of 0021
Batch size: 28 GB
Extracting batch 0008 of 0021
Batch size: 28 GB
Extracting batch 0009 of 0021
Batch size: 28 GB
Extracting batch 0010 of 0021
Batch size: 28 GB
Extracting batch 0011 of 0021
Batch size: 2

In [96]:
outputs = ui.load_outputs(combined_dir, load_traces=True)
v,layers = ui.create_ui(outputs)
ui.add_callbacks_to_ui(v,layers,outputs,savedir=combined_dir)

Cells
Found old curated + slider-ed iscell with 22418 of 22418 marked as cells
Saving old iscell_curated to backup path /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/detection/combined-full/iscell_curated_slider_old_03-10-23_18-18-38.npy
('vmap_peak', (24.40005, 27.71572265625), 'peak_val', <function add_curation_sliders.<locals>.<lambda> at 0x7f2d41320670>)
